# CT-1 Training Notebook

## 1. Introduction

## 2. Training Data Pre-processing

In [1]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def image_preprocessing(image,size,debug=False):
    image = tf.io.read_file(image) # read image file
    image = tf.image.decode_jpeg(image, channels=3) # decode image
    image = tf.image.resize(image, (size, size)) # resize image to 224x224
    image = image / 255.0 # normalize image
    image = tf.image.rgb_to_grayscale(image) # convert to grayscale
    image = tf.cast(image * 255.0, tf.uint8) # convert to uint8
    # image = tf.cast(image, tf.float32)

    # debugging mode prints the image to a file in test folder
    if debug:
        image = tf.io.encode_jpeg(image) # encode image
        # Write the encoded JPEG data to a file
        with open("test/test_result.jpg", "wb") as f:
            f.write(image.numpy())
        return
    return image

# test the image preprocessing function
image = image_preprocessing('data/1.jpg', 224, debug=True)


In [1]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load pre-trained Xception model (without top layers)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)  # Add additional dense layers if needed
predictions = Dense(num_classes, activation='softmax')(x)  # 'num_classes' is the number of classes in your dataset

# Create the model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
model.fit(train_dataset,
          epochs=10,
          validation_data=val_dataset)


test


## References
* https://www.analyticsvidhya.com/blog/2020/08/image-augmentation-on-the-fly-using-keras-imagedatagenerator/
* https://mobidev.biz/blog/few-shot-learning-explained-examples-applications-research
* https://keras.io/guides/transfer_learning/